In [ ]:
addprocs(8)

@everywhere using Fussy
Main.IJulia.set_current_module(Fussy)

return

In [ ]:
using FileIO
using JLD2

In [ ]:
cur_plots = []
use_latex = false

for cur_y_var in [:P_F, :cost]
    for (cur_index, cur_study) in enumerate([proteus_study, charybdis_study])
        push!(cur_plots, plot())
        for (cur_sub_index, cur_label) in enumerate([ :wall, :kink, :cost, :W_M ])
            tmp_reactors = getfield(cur_study, Symbol("$(cur_label)_reactors"))
            iszero(length(tmp_reactors)) && continue

            if cur_y_var == :P_F
                ylims = [0, 5000]
                if cur_sub_index == 1
                    dash_label = ( cur_index == 1 ) ? "B_{CS} = 20 T" : "H = 1.7"
                    use_latex && ( dash_label = latexstring(dash_label) )
                else
                    dash_label = ""
                end
            else
                ylims = [0, 0.025]
                dash_label = ""
            end
            ylims!(ylims...)
            
            if cur_index == 1
                cur_xx = map(tmp_reactor -> tmp_reactor.B_CS, tmp_reactors)
                plot!([1,1]*20,ylims, color=5, lineopacity=0.2, label=dash_label)
            else
                cur_xx = map(tmp_reactor -> tmp_reactor.H, tmp_reactors)
                plot!([1,1]*1.7,ylims, color=5, lineopacity=0.2, label=dash_label)
            end

            if cur_y_var == :P_F
                cur_yy = map(tmp_reactor -> tmp_reactor.P_F, tmp_reactors)
                ylabel!(use_latex ? L"P_F" : "P_F")
                plot!([0, maximum(cur_xx)],[1250,1250], color=5, lineopacity=0.2, label="")
            else
                cur_yy = map(tmp_reactor -> tmp_reactor.cost, tmp_reactors)
                ylabel!("cost")
            end
            
            if cur_y_var == :P_F && cur_sub_index == 1
                if cur_index == 1
                    cur_x_scatter = 20
                else
                    cur_x_scatter = 1.7
                end
                
                scatter!([cur_x_scatter], [1250], color=6, label="")
            end
            
            if cur_sub_index < 3 
                cur_width = 2
                cur_style = :dash
            else
                cur_width = 1
                cur_style = :solid
            end
            
            if cur_y_var == :P_F
                if cur_index == 1
                    title!("Proteus (Pulsed)")
                else
                    title!("Charybdis (Steady)")
                end
                plot!(cur_xx, cur_yy, label="", color=cur_sub_index, lineopacity=0.6, width=cur_width, style=cur_style)
            else
                if cur_index == 1
                    xlabel!(use_latex ? L"B_{CS}" : "B_CS")
                else
                    xlabel!("H")
                end
                plot!(cur_xx, cur_yy, label=use_latex ? latexstring(cur_label) : cur_label, color=cur_sub_index, lineopacity=0.6, width=cur_width, style=cur_style)
            end
            xlims!(0, maximum(cur_xx))
        end
    end
end
actual_plot = plot(cur_plots...)
# savefig(actual_plot, "pulsed_vs_steady")
# plot!()

In [ ]:
gr()

In [ ]:
using LaTeXStrings

In [ ]:
using Plots

In [ ]:
match(Reactor(:T_bar, deck=:charybdis, constraint=:wall, H=0.4857142857142857), :cost, 0.04)

In [ ]:
proteus_study = Study(:B_CS, B_CS=15.0, num_points=41, deck=:proteus, is_fast=false, sensitivity=1.0)


In [ ]:
charybdis_study = Study(:H, H=1.5, num_points=41, deck=:charybdis, is_fast=false, sensitivity=1.0, is_consistent=true, is_parallel = true)



In [ ]:
proto_studies = Dict(
    :charybdis => charybdis_study,
    :proteus => proteus_study,
)

cur_file = "../data/prototypes.jld2"

# cur_calibrations = nothing
# try
#     cur_calibrations = load(cur_file, "cur_calibrations")
# catch
#     cur_calibrations = Dict()
# end

# save(cur_file, Dict("cur_calibrations" => deepcopy(cur_calibrations)))

save(cur_file, Dict("proto_studies" => deepcopy(proto_studies)))


In [ ]:
cur_prototypes = ["proteus", "charybdis"]
cur_secondaries = ["kink", "wall"]

for cur_prototype in cur_prototypes
    println(cur_prototype)
    for cur_secondary in cur_secondaries
        ( cur_prototype == "charybdis" && cur_secondary == "kink" ) && continue
        println(cur_secondary)
        
        cur_key = "$(cur_prototype)_$(cur_secondary)"
        haskey(cur_calibrations, cur_key) && continue
        
        cur_is_consistent = ( cur_prototype == "charybdis" )
        cur_calibrations[cur_key] = Calibration(
            cur_power_list; 
            deck=Symbol(cur_prototype), 
            matched_constraint=Symbol(cur_secondary),
            is_consistent=cur_is_consistent, 
            constraint=:beta
        )
    end
    println("done.\n")
end


In [ ]:
cur_prototypes = ["proteus", "charybdis"]
cur_secondaries = ["kink", "beta", "wall"]

cur_costs = collect(10:10:100)

for cur_prototype in cur_prototypes
    println(cur_prototype)
    
    for cur_secondary in cur_secondaries
        println(cur_secondary)
        
        ( cur_prototype == "charybdis" && cur_secondary == "kink" ) && continue
        tmp_power_list = cur_power_list
        
        for cur_cost in cur_costs
            println(cur_cost) 
            
            cur_key = "$(cur_prototype)_$(cur_secondary)_cost_$(lpad(cur_cost,3,0))"
            haskey(cur_calibrations, cur_key) && continue
            
            cur_is_consistent = ( cur_prototype == "charybdis" )
            cur_calibrations[cur_key] = Calibration(
                tmp_power_list; 
                deck=Symbol(cur_prototype), 
                matched_constraint=:cost, 
                constraint_value=(cur_cost/10000),
                is_consistent=cur_is_consistent, 
                constraint=Symbol(cur_secondary)
            )
            
            sleep(0.1)
        end
        
    end
    
    println("done.\n")
    save(cur_file, Dict("cur_calibrations" => deepcopy(cur_calibrations)))
end


In [ ]:
for cur_proto in ["proteus"]#, "charybdis"]
    for cur_milli_cost in [7] # 1:2:15
        cur_name = "$(cur_proto)_cost_$(cur_milli_cost)"
        println(cur_name)
        
        if !haskey(cur_calibrations, cur_name)
            println(cur_milli_cost/1000)
            cur_calibrations[cur_name] = Calibration(cur_power_list; deck=Symbol(cur_proto), constraint=:beta, matched_constraint=:cost, constraint_value=cur_milli_cost/1000)
        end 
        
        break
    end
end

In [ ]:
save(cur_file, Dict("cur_calibrations" => cur_calibrations))


In [ ]:


println("charybdis_H")
# if !haskey(cur_calibrations, "charybdis_H")
    cur_calibrations["charybdis_H"] = Calibration(cur_power_list; deck=:charybdis, is_consistent=true)
# end

println("scylla_B_CS")
# if !haskey(cur_calibrations, "scylla_B_CS")
    cur_calibrations["scylla_B_CS"] = Calibration(cur_power_list; deck=:scylla, H=1.0, constraint=:beta)
# end

println("scylla_B_CS_Kink")
# if !haskey(cur_calibrations, "scylla_B_CS_Kink")
    cur_calibrations["scylla_B_CS_Kink"] = Calibration(cur_power_list; deck=:scylla, H=1.0, constraint=:beta, matched_constraint=:kink)
# end

save(cur_file, Dict("cur_calibrations" => cur_calibrations))

return

In [ ]:
cur_calibrations

In [ ]:
save(cur_file, Dict("cur_calibrations" => cur_calibrations))

In [ ]:
using Plots

In [ ]:
map(xx->xx.B_CS,cur_calibrations["scylla_B_CS_B_0_4_5"].reactors)

In [ ]:

using LaTeXStrings
Plots.pgfplots()

min_P_F = minimum(map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS"].reactors))
max_P_F = maximum(map(cur_reactor -> cur_reactor.P_F, cur_calibrations["charybdis_H"].reactors))

return

In [ ]:
cur_scylla_plot = plot()

cur_x_w = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS"].reactors)
cur_y_w = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS"].reactors)

plot!(cur_x_w, cur_y_w, label="Beta-Wall Limit")

cur_x_k = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS_Kink"].reactors)
cur_y_k = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS_Kink"].reactors)

plot!(cur_x_k, cur_y_k, label="Beta-Kink Limit")

# cur_x_b_1 = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS_B_0_4_5"].reactors)
# cur_y_b_1 = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS_B_0_4_5"].reactors)

# plot!(cur_x_b_1, cur_y_b_1, label=L"B_0 = 4.5", style=:dash)

# cur_x_b_2 = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS_B_0_5_5"].reactors)
# cur_y_b_2 = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS_B_0_5_5"].reactors)

# plot!(cur_x_b_2, cur_y_b_2, label=L"B_0 = 5.5", style=:dash)

# cur_x_b_3 = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS_B_0_6_5"].reactors)
# cur_y_b_3 = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS_B_0_6_5"].reactors)

# plot!(cur_x_b_3, cur_y_b_3, label=L"B_0 = 6.5", style=:dash)

# scatter!(cur_x, cur_y, label="")

# ylims!(0,50)
# xlims!(0,3000)

# xlabel!(L"P_F")
# ylabel!(L"B_{CS}")
# title!("Scylla Prototyping")

# plot!([min_P_F, max_P_F, max_P_F, min_P_F, min_P_F], [50,50,0,0,50], fill=true, fillopacity=0.1, color=3, label="Valid Region")

# tmp_p = 2250
# tmp_b = polyfit(cur_x, cur_y)(tmp_p)

# plot!([tmp_p,tmp_p],[0,50], label=latexstring("P_F = $tmp_p"), color=2)
# scatter!([tmp_p], [tmp_b], color=2, label=latexstring("B_{CS} = $(round(tmp_b,2))"))


# scatter!(cur_x, cur_y, label="")

ylims!(0,50)
xlims!(0,3000)

plot!([0,3000],[25,25], style=:dot, color=:black, opacity=0.75, label=L"B_{CS} = 25 \, \textnormal{T}")

xlabel!(L"P_F")
ylabel!(L"B_{CS}")
title!("Scylla Prototyping")

scatter!([1000], [25], label=L"P_F = 1000 \, \textnormal{MW}")
scatter!([2250], [25], label=L"P_F = 2250 \, \textnormal{MW}")

# plot!([min_P_F, max_P_F, max_P_F, min_P_F, min_P_F], [50,50,0,0,50], fill=true, fillopacity=0.1, color=3, label="Valid Region")


In [ ]:
cur_x = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["charybdis_H"].reactors)
cur_y = map(cur_reactor -> cur_reactor.H, cur_calibrations["charybdis_H"].reactors)

cur_charybdis_plot = plot(cur_x, cur_y, label="")

ylims!(0,2)
xlims!(0,3000)

xlabel!(L"P_F")
ylabel!(L"H")
title!("Charybdis Prototyping")

plot!([1000, 2250, 2250, 1000, 1000], [2,2,0,0,2], fill=true, fillopacity=0.1, color=3, label="Valid Region")


In [ ]:
savefig(cur_charybdis_plot, "../images/prototype_charybdis.tex")
savefig(cur_scylla_plot, "../images/prototype_scylla.tex")


In [ ]:
cur_x = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["scylla_B_CS_Kink"].reactors)
cur_y = map(cur_reactor -> cur_reactor.B_CS, cur_calibrations["scylla_B_CS_Kink"].reactors)

cur_scylla_plot = plot(cur_x, cur_y, label="")
scatter!(cur_x, cur_y, label="")

ylims!(0,50)
xlims!(0,3000)

xlabel!(L"P_F")
ylabel!(L"B_{CS}")
title!("Scylla Prototyping")

plot!([min_P_F, max_P_F, max_P_F, min_P_F, min_P_F], [50,50,0,0,50], fill=true, fillopacity=0.1, color=3, label="Valid Region")

tmp_p = 2250
tmp_b = polyfit(cur_x, cur_y)(tmp_p)

# plot!([tmp_p,tmp_p],[0,50], label=latexstring("P_F = $tmp_p"), color=2)
# scatter!([tmp_p], [tmp_b], color=2, label=latexstring("B_{CS} = $(round(tmp_b,2))"))


In [ ]:
cur_calibrations["scylla_cost_13"].reactors

In [ ]:
work_x = []
work_y = []
work_z = []

cur_scylla_plot = plot()

cur_keys = sort(collect(keys(cur_calibrations)))

for cur_key in cur_keys
    cur_value = cur_calibrations[cur_key]
    
    startswith(cur_key, "scylla_") || continue
    contains(cur_key, "_25") && continue
    
    contains(cur_key, "_B_0_") && continue
    
    tmp_x = []
    tmp_y = []
    tmp_z = []
    for work_reactor in cur_value.reactors
        push!(tmp_x, work_reactor.P_F)
        push!(tmp_y, work_reactor.B_CS)
        push!(tmp_z, work_reactor.B_0)
    end
    
    if endswith(cur_key,"Kink") 
        cur_color = 1
        cur_label = "Beta-Kink"
    else
        cur_color = 3
        cur_label = "Beta-Wall"
    end
    
    plot!(tmp_x, tmp_y, width=2,color=cur_color,label=cur_label)

    append!(work_x, tmp_x)
    append!(work_y, tmp_y)
    append!(work_z, tmp_z)
end

work_min_y = minimum(work_y)

cur_colors = []

cur_dict = OrderedDict()

work_x = []
work_y = []
work_z = []

for cur_key in cur_keys
    cur_value = cur_calibrations[cur_key]
    
    startswith(cur_key, "scylla_") || continue
    contains(cur_key, "_25") && continue
    
    contains(cur_key, "_B_0_") && continue
  
    ( cur_key == "scylla_B_CS_B_0_4_0" ) && continue
    
    tmp_x = []
    tmp_y = []
    tmp_z = []
    for work_reactor in cur_value.reactors
        push!(tmp_x, work_reactor.P_F)
        push!(tmp_y, work_reactor.B_CS)
        push!(tmp_z, work_reactor.B_0)
    end
    
    isempty(tmp_z) && println(cur_key)
    isempty(tmp_z) && continue

    append!(work_x, tmp_x)
    append!(work_y, tmp_y)
    append!(work_z, tmp_z)
    
    cur_dict[cur_key] = (tmp_x, tmp_y, tmp_z)
end

work_min_y = min(work_min_y, minimum(work_y))

cur_index = 0
for (cur_key, (tmp_x, tmp_y, tmp_z)) in cur_dict
    cur_index += 1 
    
    cur_scalar = (cur_index-1)/(length(cur_dict)-1)
    cur_scalar *= 0.5
    cur_scalar += 0.25
    println(cur_scalar)
    
    cur_color = get(ColorSchemes.viridis, cur_scalar)
    push!(cur_colors, cur_color)
    
    plot!(tmp_x, tmp_y, label="$(round(tmp_z[1],1))", color=cur_color, label="")
    
end

# cur_spline = Spline2D(work_x, work_y, work_z, s=1e2)

ylims!(0,40)
# xlims!(0,3000)
# new_x = linspace(0, 3000, 41)
# new_y = linspace(0, 50, 41)
# contour!(new_x, new_y, new_z)
plot!()

ylabel!(L"B_{CS}")
xlabel!(L"P_F")

work_min_z = minimum(work_z)
work_max_z = maximum(work_z)

cur_diff_z = work_max_z - work_min_z

work_min_z -= 0.25 * cur_diff_z
work_max_z += 0.25 * cur_diff_z

cur_z_list = collect(linspace(work_min_z, work_max_z, length(cur_dict)+2))

contour!(
    cur_z_list * ones(length(cur_dict)+2)',
    color=:viridis,colorbar=true, fillopacity=0, colorbar_title=L"B_0")
plot!()

plot!([0, work_max_z, work_max_z, 0, 0]*2, [0, 0, work_max_z, work_max_z,0]*2, fill=true, color=:white, label="")

work_min_y = 20
plot!([0,4500],[work_min_y,work_min_y],style=:dash, label="$(Int(round(work_min_y,0)))\\,\\,\\textnormal{T}", color=4)

scatter!([1986], [20], label=L"2000 \, \textnormal{MW}", color=:yellow, marker=:diamond, markersize=8)
# scatter!([1050], [20])
scatter!([1658], [20], label=L"1700 \, \textnormal{MW}", color=1)
scatter!([2538], [20], label=L"2500 \, \textnormal{MW}",color=3)

title!("Scylla Prototyping")

xlims!(0,4650)


In [ ]:
using Plots
pgfplots()

using LaTeXStrings
using ColorSchemes

In [ ]:
cur_x = map(cur_reactor -> cur_reactor.P_F, cur_calibrations["charybdis_H"].reactors)
cur_y = map(cur_reactor -> cur_reactor.H, cur_calibrations["charybdis_H"].reactors)

cur_charybdis_plot = plot(cur_x, cur_y, label="Beta-Wall", color=2)

ylims!(0,3)
xlims!(0,4650)

xlabel!(L"P_F")
ylabel!(L"H")
title!("Charybdis Prototyping")

plot!([1658,1658],[0,3],color=1,width=2,label=L"1700 \, \textnormal{MW}")
plot!([2538,2538],[0,3],color=3,width=2,label=L"2500 \, \textnormal{MW}")

# plot!([1000, maximum(cur_x), maximum(cur_x), 1000, 1000], [2,2,0,0,2], fill=true, fillopacity=0.1, color=3, label="Valid Region")

plot!([2000,2000],[0,3], color=4, style=:dash,label=L"2000 \, \textnormal{MW}")
scatter!([2000],[1.565775], color=:yellow, marker=:diamond, markersize=8, label="H = 1.57")

scatter!([1658],[1.62], color=1, label="H = 1.62")
scatter!([2538],[1.495], color=3, label="H = 1.50")

scatter!([minimum(cur_x)],[maximum(cur_y)], label="")


In [ ]:
pgfplots()

In [ ]:
match(Reactor(:T_bar, deck=:scylla, constraint=:beta, B_CS=20.34), :B_0, 5.5).P_F

In [ ]:
savefig(cur_charybdis_plot, "../images/prototype_charybdis.jpg")
savefig(cur_scylla_plot, "../images/prototype_scylla.jpg")

In [ ]:
cur_calibrations

In [ ]:
using Plots

In [ ]:
cur_plots = []
for cur_deck in ["proteus", "charybdis"]
    push!(cur_plots, plot())

    if cur_deck == "proteus"
        ylims!(0,40)
    else
        ylims!(0,3)
    end

    xlims!(0,5000)
    
    for (cur_key, cur_value) in sort(cur_calibrations)
        startswith(cur_key, cur_deck) || continue
        contains(cur_key, "_B_0_") && continue
        
        tmp_reactors = deepcopy(cur_calibrations[cur_key].reactors)
        filter!(tmp_reactor -> tmp_reactor.is_good && tmp_reactor.is_valid, tmp_reactors)
        
        cur_x = map(cur_reactor -> cur_reactor.P_F, tmp_reactors)
        isempty(cur_x) && continue
        
        if cur_deck == "proteus"
            cur_y = map(cur_reactor -> cur_reactor.B_CS, tmp_reactors)
        else
            cur_y = map(cur_reactor -> cur_reactor.H, tmp_reactors)
        end

        if !contains(cur_key, "cost")
            plot!(cur_x, cur_y, width=2)
        else
            scatter!(cur_x, cur_y, style=:dash, label=replace(cur_key, "_", " "))
        end
        
        println(cur_key) 
    end
    plot!(legend=false)
end
plot(cur_plots...)

In [ ]:
plotlyjs()

In [ ]:
cur_calibrations[]

In [ ]:
map(xx->xx.cost, cur_calibrations["scylla_B_CS_Kink"].reactors)

In [ ]:
map(xx->xx.cost, cur_calibrations["charybdis_cost_15"].reactors)


In [ ]:
for (cur_key, cur_value) in cur_calibrations
    contains(cur_key, "_cost_") || continue 
    
    first_part, last_part = split(cur_key, "_cost_")
    last_part = rpad(last_part, 3, "0")
    new_key = "$(first_part)_cost_$(last_part)"
    
    cur_calibrations[new_key] = cur_value
    delete!(cur_calibrations, cur_key)
end

In [ ]:
sort(collect(keys(cur_calibrations)))

In [ ]:
proto_studies

In [ ]:
cur_calibrations